In [3]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib


In [4]:
#Carregamento e visualização inicial dos dados

df = pd.read_csv("base_viabilidade.csv", delimiter=";")
df = df.drop(columns=["dt_manut"])
# Visualizando as primeiras linhas
df.head()

,tipo_manut,tempo_manut,tec_resp,POP_utiliz,problema_recorr,falha_crit,tempo_resolucao_padr,status_manut
0,Corretiva,180,Técnico A,POP1,Sim,Não,150,Concluído
1,Preventiva,120,Técnico B,POP2,Não,Não,100,Concluído
2,Corretiva,90,Técnico C,POP3,Sim,Sim,120,Pendente
3,Preventiva,75,Técnico D,POP4,Não,Não,80,Concluído
4,Corretiva,200,Técnico A,POP1,Sim,Sim,180,Pendente


In [6]:
#  Renomeando as colunas para facilitar o uso
df.columns = [
     'tipo_manutencao', 'tempo_estimado', 'tecnico', 'pop',
    'necessidade_pecas', 'problema_reincidente',
    'tempo_real_manutencao', 'status'
]

# Visualizando novamente
df.head()


,tipo_manutencao,tempo_estimado,tecnico,pop,necessidade_pecas,problema_reincidente,tempo_real_manutencao,status
0,Corretiva,180,Técnico A,POP1,Sim,Não,150,Concluído
1,Preventiva,120,Técnico B,POP2,Não,Não,100,Concluído
2,Corretiva,90,Técnico C,POP3,Sim,Sim,120,Pendente
3,Preventiva,75,Técnico D,POP4,Não,Não,80,Concluído
4,Corretiva,200,Técnico A,POP1,Sim,Sim,180,Pendente


In [7]:
#  Verificando informações gerais e tipos de dados
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   tipo_manutencao        60 non-null     object
 1   tempo_estimado         60 non-null     int64 
 2   tecnico                60 non-null     object
 3   pop                    60 non-null     object
 4   necessidade_pecas      60 non-null     object
 5   problema_reincidente   60 non-null     object
 6   tempo_real_manutencao  60 non-null     int64 
 7   status                 60 non-null     object
dtypes: int64(2), object(6)
memory usage: 3.9+ KB


In [8]:
# Convertendo colunas categóricas em valores numéricos
# Usaremos LabelEncoder para variáveis categóricas
colunas_categoricas = [
    'tipo_manutencao', 'tecnico', 'pop',
    'necessidade_pecas', 'problema_reincidente'
]

le = LabelEncoder()

for coluna in colunas_categoricas:
    df[coluna] = le.fit_transform(df[coluna])

df.head()


,tipo_manutencao,tempo_estimado,tecnico,pop,necessidade_pecas,problema_reincidente,tempo_real_manutencao,status
0,0,180,0,0,1,0,150,Concluído
1,1,120,1,1,0,0,100,Concluído
2,0,90,2,2,1,1,120,Pendente
3,1,75,3,3,0,0,80,Concluído
4,0,200,0,0,1,1,180,Pendente


In [9]:
# Definindo as features (X) e o alvo (y)
X = df[[
    'tipo_manutencao', 'tempo_estimado', 'tecnico', 'pop',
    'necessidade_pecas', 'problema_reincidente'
]]

y = df['tempo_real_manutencao']


In [10]:
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [11]:
# Treinando três modelos e avaliando desempenho
modelos = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Linear Regression': LinearRegression(),
    'SVR': SVR()
}

resultados = {}

for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    resultados[nome] = {
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R²': r2_score(y_test, y_pred)
    }

# Mostrando os resultados
for nome, metricas in resultados.items():
    print(f"\nModelo: {nome}")
    for metrica, valor in metricas.items():
        print(f"{metrica}: {valor:.2f}")



Modelo: Random Forest
MAE: 3.40
RMSE: 4.42
R²: 0.99

Modelo: Linear Regression
MAE: 5.33
RMSE: 7.03
R²: 0.98

Modelo: SVR
MAE: 35.60
RMSE: 37.81
R²: 0.31


In [12]:
#Treinando o modelo final com todos os dados
# Escolha aqui o melhor modelo (ex: Random Forest)
modelo_final = RandomForestRegressor(random_state=42)
modelo_final.fit(X, y)


RandomForestRegressor(random_state=42)

In [13]:
# Salvando (serializando) o modelo com Joblib
joblib.dump(modelo_final, 'modelo_final.pkl')
print("Modelo salvo como 'modelo_final.pkl'")


Modelo salvo como 'modelo_final.pkl'
